## Dependencies and Setup

Load the target image, and a pair of simulations to compare

In [ ]:
using Revise
using Paint
using Serialization
using Images, ImageShow
using Plots
using Evolutionary
using BlackBoxOptim
using StaticArrays
using ImageFeatures
using IntervalSets
using ReinforcementLearning
using ReinforcementLearningBase
using ReinforcementLearningZoo
using Combinatorics
using StatsBase
using Random

In [ ]:
target = float.(load("../lisa.png"))
hist = Serialization.deserialize("../output/simresult/simlog_100-prims_100-batch_10-epoch_100-refine.bin")
state = hist.history[end]
nothing

In [ ]:
function drawstate(state, size)
    initial = zeros(RGB{Float32}, size) .+ state.background
    for k in eachindex(state.shapes)
        draw!(initial, state.shapes[k], RGBA{Float32}(state.colours[k].r, state.colours[k].g, state.colours[k].b, state.alpha), RasterAlgorithmScanline())
    end
    initial
end

In [ ]:
drawstate(state, size(target))

In [ ]:
drawstate(state, size(target) .* 2)

In [ ]:
makecol(x) = RGBA{Float32}(x.r, x.g, x.b, state.alpha)

In [ ]:
background

In [ ]:
foreground

In [ ]:
@time for round = 1:1000
    Threads.@threads for which = 1:100
        background = zero(target) .+ state.background
        for i = 1:(which-1)
            draw!(background, state.shapes[i], makecol(state.colours[i]), RasterAlgorithmScanline())
        end
        
        foreground = zeros(RGBA{Float32}, size(target))
        for i = (which+1):length(state.shapes)
            draw!(foreground, state.shapes[i], makecol(state.colours[i]), RasterAlgorithmScanline())
        end
        
        best = 0.0f0
        delta = zeros(Float32, 6)
        bestshape = state.shapes[which]
        for k = 1:10
            newshape = mutate(bestshape, delta .* 0.01f0)
            ld = drawloss(target, background, newshape, makecol(state.colours[which]), SELoss(), RasterAlgorithmScanline(), foreground = foreground)
            if ld < best
                best = ld
                bestshape = newshape
            end
            randn!(delta)
        end
        
        state.shapes[which] = bestshape
    end
end

state.colours, state.background = alpharecolor(target, state.shapes, state.alpha, RasterAlgorithmScanline())

In [ ]:
imloss(drawstate(state, size(target)), target, SELoss())

In [ ]:
drawstate(state, size(target))

In [ ]:
img = ones(RGB{Float32}, (200, 200))
tri1 = Triangle(Point(0.2, 0.4), Point(0.7, 0.2), Point(0.9, 0.9))
draw!(img, tri1, RGB{Float32}(1, 0, 0), RasterAlgorithmScanline())
img

In [ ]:
function disp(tri)
    newimg = copy(img)
    draw!(newimg, tri, RGBA{Float32}(0, 0, 1, 0.5), RasterAlgorithmScanline())
    newimg
end

In [ ]:
blank = ones(RGB{Float32}, (200, 200))
base = imloss(blank, img, SELoss())
function err(tri)
    col = averagepixel(img, blank, 1.0, tri, RasterAlgorithmScanline())
    base + drawloss(img, blank, tri, col, SELoss(), RasterAlgorithmScanline())
end

In [ ]:
optimtribase = Triangle(Point(0.4, 0.4), Point(0.5, 0.6), Point(0.6, 0.4))
optimtri = optimtribase
println(err(optimtri))
disp(optimtri)

In [ ]:
losses = []
@time for i = 1:1000
    which = rand(1:6)
    deltas = zeros(6)
    # deltas[which] = randn(Float32) .* 0.005f0
    deltas = randn(Float32, 6) .* 0.005f0
    newt = mutate(optimtri, deltas)
    if err(newt) < err(optimtri)
        optimtri = newt
    end
    push!(losses, err(optimtri))
end
println(err(optimtri))
disp(optimtri)

In [ ]:
rand(1:6)

In [ ]:
optimtri

In [ ]:
tri1

In [ ]:
using Evolutionary

In [ ]:
function sample_loss(xs)
    tri = Triangle(SVector{6, Float32}(xs))
    Float64(err(tri))
end

In [ ]:
Evolutionary.optimize(sample_loss, [0.4, 0.4, 0.5, 0.6, 0.6, 0.4], Evolutionary.DE(populationSize=2000))

In [ ]:
optimtribase

In [ ]:
optres = bboptimize(x -> sample_loss(x); SearchRange=(0, 1), MaxTime=30, NumDimensions=6, PopulationSize=2000)
nothing

In [ ]:
rngs = 2.0f0 .* rand(Float32, 20, 6) .- 0.5f0
tris = [Triangle(SVector{6, Float32}(slice)) for slice in eachslice(rngs, dims=1)]
cols = [RGBA{Float32}(slice...) for slice in eachslice(rand(20, 4), dims=1)]

In [ ]:
img = ones(RGB{Float32}, (200, 200))
for k = 1:20
    draw!(img, tris[k], cols[k], RasterAlgorithmScanline())
end
img

In [ ]:
optimtribase = Triangle(Point(0.4, 0.4), Point(0.5, 0.6), Point(0.6, 0.4))
optimtri = optimtribase
println(err(optimtri))
disp(optimtri)

In [ ]:
# losses = []
best = err(optimtri)
@time for i = 1:1
    deltas = randn(Float32, 6) .* 0.005f0
    newt = mutate(optimtri, deltas)
    @time ev = err(newt)
    if ev < best
        optimtri = newt
        best = ev
    end
end

println(err(optimtri))
disp(optimtri)

In [ ]:
optres = bboptimize(x -> sample_loss(x); SearchRange=(-1, 2), MaxTime=3, NumDimensions=6, PopulationSize=50)
nothing

In [ ]:
disp(Triangle(SVector{6, Float32}(optres.archive_output.best_candidate)))

In [ ]:
img

In [ ]:
optres.parameters.dicts[2]

In [ ]:
@time begin
nbatch = 1
rngs = 2.0f0 .* rand(nbatch, 6) .- 0.5f0
pop = [collect(slice) for slice in eachslice(rngs, dims=1)]
tris = collect(map(Triangle, map(SVector{6, Float32}, pop)))
cols = averagepixel_batch(img, blank, 1.0, tris, RasterAlgorithmScanline())
losses = drawloss_batch(img, blank, tris, cols, SELoss(), RasterAlgorithmScanline())

for k=1:1000
    rngs = randn(Float32, nbatch, 6) .* 0.01f0
    newtris = mutate_batch(tris, rngs)
    newcolours = averagepixel_batch(img, blank, 1.0, newtris, RasterAlgorithmScanline())
    newlosses = drawloss_batch(img, blank, newtris, newcolours, SELoss(), RasterAlgorithmScanline())
    for i=1:nbatch
        if newlosses[i] < losses[i]
            losses[i] = newlosses[i]
            tris[i] = newtris[i]
            pop[i] = [tris[i].vertices[1][1], tris[i].vertices[1][2], tris[i].vertices[2][1], tris[i].vertices[2][2], tris[i].vertices[3][1], tris[i].vertices[3][2]]
            cols[i] = newcolours[i]
        end
    end
end

end

println(minimum(losses) + base)
# pop = pop[sortperm(losses)[1:50]]
# sort(losses)[1:50] .+ base

In [ ]:
optres = bboptimize(x -> sample_loss(x), pop; SearchRange=(-2, 3), MaxTime=1, NumDimensions=6)
nothing

In [ ]:
target = float.(load("../lisa.png"))
draw!(target, Triangle(Point(0.45, 0.45), Point(0.5, 0.515), Point(0.515, 0.45)), one(RGB{Float32}), RasterAlgorithmScanline())
img = target